# Testing entity.jl

This notebook is dedicated to experimenting with a new structure called Entity, designed to encapsulate metadata using HllSets. This initiative builds on the innovative work by Mike Saint-Antoine of SimpleGrad.jl, who adapted Andrey Karpathy's foundational MicroGrad project. While the original frameworks by Mike S-A and Andrey K utilize Numbers as the fundamental components of their neural networks, our approach replaces Numbers with HllSets.

It is often argued that Numbers in conventional neural network models are derived by transforming "real" entities through numerical embedding. This method is effective for those entities. However, we aim to shift our focus from data to metadata, thereby describing entities more abstractly. A significant advantage of using metadata is its inherent ability to categorically separate entities into semantically similar groups. These groups are not strictly distinct; the sets of entities described by different metadata often overlap.

The most crucial aspect for us is that each piece of metadata correlates with a specific set of entities. In the realm of metadata, an HllSet serves as an embedding for a collection of entities. This embedding is represented not as a numerical value but as a fixed-size bit-vector, specifically a 2-dimensional Tensor (64, P). In Julia, this is expressed as Vector{BitVector}, where each vector has a fixed length of 64 bits and the number of these bit-sets is determined by P. The parameter P defines the precision of the HyperLogLog approximation of the collection of entities.


In [1]:
include("src/sgs_entity.jl")

using .Entity
using .HllSets

using Random
using Base
using JSON3

using PyCall
# Default Redis instance
redis = pyimport("redis")
r = redis.Redis(host="localhost", port=6379)

using Serialization


In [2]:
# Initialize test HllSets
hll1 = HllSets.HllSet{10}()
hll2 = HllSets.HllSet{10}()
hll3 = HllSets.HllSet{10}()
hll4 = HllSets.HllSet{10}()
hll5 = HllSets.HllSet{10}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
HllSets.add!(hll1, s1)
HllSets.add!(hll2, s2)
HllSets.add!(hll3, s3)
HllSets.add!(hll4, s4)
HllSets.add!(hll5, s5)

In [3]:
JSON3.write(["nothing"])

"[\"nothing\"]"

In [4]:
entity1 = Entity.Instance{10}(r, hll1)
entity2 = Entity.Instance{10}(r, hll2)
Entity.isequal(entity1, entity2)

# Access the type parameter P
P_type = typeof(entity1).parameters[1]

println("The type parameter P is: ", P_type)

entity1

["nothing"]
["nothing"]
The type parameter P is: 10


Instance{10}("0cecf3c15b70ae8593d6d4b04bbc78ce902181a4", HllSet{10}()
, 0.0, nothing)

In [6]:
c = Entity.union(r, entity1, entity2)
c
println(c)

["0cecf3c15b70ae8593d6d4b04bbc78ce902181a4","7d956da33f2ebe56cf59aff16ef38b49990394e3"]
Instance{10}("80ed0fb6eb3ffdccf1c806fb946806c483d9c52d", HllSet{10}()
, 0.0, Main.Entity.Operation{typeof(Main.Entity.union), 10}(Main.Entity.union, HllSet{10}[HllSet{10}()
, HllSet{10}()
]))


In [ ]:
println("Entity 1: ", HllSets.count(entity1.hll))

In [ ]:
json = Entity.retrieve_entity("e95126675977cb59b1713e6c0e8ff45e5a5830b2", prefix="b")
# println(length(json["dataset"]))

In [ ]:
println(c.op)
Entity.backprop!(c, c.op)
println(c.op)
c

In [ ]:
d = Entity.intersect(entity1, entity2)

println(HllSets.count(d.hll))

In [ ]:
println(d.op)
Entity.backprop!(d, d.op)
println(d.op)
d

In [ ]:
e = Entity.xor(entity1, entity2)


println(HllSets.count(e.hll))

In [ ]:
println(e.op)
Entity.backprop!(e, e.op)
println(e.op)
e

In [ ]:
n, rem, del = Entity.diff(c,d)
println(del)
println(rem)
println(n)


In [ ]:
f = Entity.advance(c,d)

println(f)

In [ ]:
g = Entity.advance(d,f)
println(f.op.args[3])

In [ ]:
h = Entity.advance(f,g)

println(h.op.args[1])

In [ ]:
using Random

function remove_random_bits(bitvectors::Vector{BitVector}, N::Int)
    # Filter out empty BitVectors
    non_empty_bitvectors = filter(bv -> !isempty(bv), bitvectors)
    
    total_bits = sum(count(b -> b, bv) for bv in non_empty_bitvectors)
    if N > total_bits
        error("N is greater than the total number of true bits in the BitVectors")
    end

    for _ in 1:N
        # Randomly select a non-empty BitVector
        bv_index = rand(1:length(non_empty_bitvectors))
        bv = non_empty_bitvectors[bv_index]

        # Get indices with true values
        true_indices = findall(bv)

        # Randomly select one of the true indices
        if !isempty(true_indices)
            bit_index = rand(true_indices)

            # Set the selected bit to false
            bv[bit_index] = false
        end
    end

    return bitvectors
end

# Example usage
bitvectors = [BitVector([true, false, true]), BitVector([true, true, false]), BitVector([false, true, true]), BitVector([])]
N = 3
modified_bitvectors = remove_random_bits(bitvectors, N)
println(modified_bitvectors)